In [1]:
from llmcoder.eval.metrics.extrinsic import levenshtein_distance_score, bleu_score, trf_similarity_score, sequence_matcher_score, gpt_reviewer_score


/home/psaegert/miniconda3/envs/llmcoder/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
fib1 = """
# Function that returns the nth Fibonacci number
def fibonacci_recursive(n):
    if n <= 0:
        return []
    elif n == 1:
        return [0]
    elif n == 2:
        return [0, 1]
    else:
        sequence = fibonacci_recursive(n-1)
        sequence.append(sequence[-1] + sequence[-2])
        return sequence
"""

In [3]:
fib2 = """
# Function that returns the nth Fibonacci number
def fibonacci_iterative(n):
    if n <= 0:
        return []
    elif n == 1:
        return [0]
    sequence = [0, 1]
    for i in range(2, n):
        sequence.append(sequence[i-1] + sequence[i-2])
    return sequence
"""

In [4]:
fib3 = """
# Function that returns the nth Fibonacci number
def fibonacci_memoization(n, memo = {0: 0, 1: 1}):
    if n not in memo:
        memo[n] = fibonacci_memoization(n-1, memo) + fibonacci_memoization(n-2, memo)
    return [memo[i] for i in range(n)]
"""

In [5]:
geo1 = """
# Function that returns the nth Fibonacci number
import math

def geometric_mean(numbers):
    if not numbers:
        return 0

    product = 1
    for number in numbers:
        product *= number

    return math.pow(product, 1/len(numbers))
"""

In [6]:
scores_list = [
    ('levenshtein', levenshtein_distance_score),
    ('bleu', bleu_score),
    ('trf', trf_similarity_score),
    ('seq', sequence_matcher_score),
    ('gpt', gpt_reviewer_score)
]

In [7]:
# Take fib1 as ground truth and compare the similarity of fib1 to fib2 and fib3 and geo1

for name, score_func in scores_list:
    print(name)
    print(f'fib1 vs fib1: {score_func(fib1, fib1):.3f}')
    print(f'fib1 vs fib2: {score_func(fib1, fib2):.3f}')
    print(f'fib1 vs fib3: {score_func(fib1, fib3):.3f}')
    print(f'fib1 vs geo1: {score_func(fib1, geo1):.3f}')

levenshtein
fib1 vs fib1: 0.000
fib1 vs fib2: 79.000
fib1 vs fib3: 184.000
fib1 vs geo1: 186.000
bleu
fib1 vs fib1: 1.000
fib1 vs fib2: 0.752
fib1 vs fib3: 0.397
fib1 vs geo1: 0.379
trf
fib1 vs fib1: 1.000
fib1 vs fib2: 0.937
fib1 vs fib3: 0.895
fib1 vs geo1: 0.820
seq
fib1 vs fib1: 1.000
fib1 vs fib2: 0.624
fib1 vs fib3: 0.329
fib1 vs geo1: 0.306
gpt
fib1 vs fib1: 0.000
fib1 vs fib2: 1.000
fib1 vs fib3: 1.000
fib1 vs geo1: -2.000
